In [1]:
!git clone https://github.com/eren-mahi/AI.git

Cloning into 'AI'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 5.17 KiB | 5.17 MiB/s, done.


In [2]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.metrics import mean_absolute_error

pd.options.mode.chained_assignment = None

dt = pd.read_csv("/content/AI/Car_Price_Prediction/car_data_100_entries.csv")

X = dt[["Ctype","Cc","Wbase","Wtype","Btype"]]
Y = dt[["Price"]]

X_Scaler = MinMaxScaler(feature_range=(0,1))
Y_Scaler = MinMaxScaler(feature_range=(0,1))

X[X.columns] = X_Scaler.fit_transform(X[X.columns])
Y[Y.columns] = Y_Scaler.fit_transform(Y[Y.columns])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

model = Sequential()
model.add(Dense(50, input_dim = 5, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(
    loss='mean_squared_error',
    optimizer='SGD'
)

model.fit(
    X_train,
    Y_train,
    epochs=50,
    batch_size=10,
    shuffle=True,
    verbose=2
)

joblib.dump(X_Scaler, "/content/AI/Car_Price_Prediction/X_Scaler.pkl")
joblib.dump(Y_Scaler, "/content/AI/Car_Price_Prediction/Y_Scaler.pkl")

model.save("/content/AI/Car_Price_Prediction/Car_Price_model.h5")

print("Model training results:")

predictions_train = model.predict(X_train, verbose=0)
mse_train = mean_absolute_error(
    Y_Scaler.inverse_transform(predictions_train),
    Y_Scaler.inverse_transform(Y_train)
)

print(f" - Training Set Error: {mse_train}")

predictions_test = model.predict(X_test, verbose=0)

mse_test = mean_absolute_error(
    Y_Scaler.inverse_transform(predictions_test),
    Y_Scaler.inverse_transform(Y_test)
)
print(f" - Test Set Error: {mse_test}")